# Automatic Evaluator

### Requirements

In [44]:
%%capture
!pip install ipywidgets
!pip install widgetsnbextension

import ipywidgets as widgets
from pathlib import Path

## Model Upload

In [33]:
uploader = widgets.Text(
    value='model.gz',
    placeholder='Model Path',
    description="Model Path:",
    disabled=False
)
display(uploader)

Text(value='model.gz', description='Model Path:', placeholder='Model Path')

In [35]:
model_path = uploader.value
model_path

'model.gz'

## Game Selection

In [54]:
games = [x.name.replace('.py', '') for x in Path('../hackatari/games').glob('**/*') if x.is_file() and x.name != '__init__.py']

game_selector = widgets.Select(
    options = games,
    value = "pong",
    description = 'Game:',
    disabled = False,
)
display(game_selector)

Select(description='Game:', index=12, options=('yarsrevenge', 'doubledunk', 'fishingderby', 'breakout', 'freew…

In [55]:
selected_game = game_selector.value

# Run Game

In [58]:
print(f"Running Game: {selected_game} with model located at: {model_path}")

Running Game: pong with model located at: model.gz


# Evaluation